In [1]:
from matplotlib.pyplot import axis
import torch
import sys
import os
import json
import time
import numpy as np
import argparse

from torch.utils.data import DataLoader
from torch.utils.data import WeightedRandomSampler
from umap.umap_ import find_ab_params

from singleVis.custom_weighted_random_sampler import CustomWeightedRandomSampler
from singleVis.SingleVisualizationModel import SingleVisualizationModel
from singleVis.losses import SingleVisLoss, UmapLoss, ReconstructionLoss
from singleVis.edge_dataset import DataHandler
from singleVis.trainer import SingleVisTrainer
from singleVis.data import DataProvider
import singleVis.config as config
from singleVis.eval.evaluator import Evaluator
from singleVis.spatial_edge_constructor import kcParallelSpatialEdgeConstructor
from singleVis.temporal_edge_constructor import GlobalTemporalEdgeConstructor

In [2]:
CONTENT_PATH = "/home/xianglin/projects/DVI_data/resnet18_cifar10"
DATASET = "cifar10_full"
GPU_ID = 0

In [3]:
LEN = config.dataset_config[DATASET]["TRAINING_LEN"]
LAMBDA = config.dataset_config[DATASET]["LAMBDA"]
L_BOUND = config.dataset_config[DATASET]["L_BOUND"]
MAX_HAUSDORFF = config.dataset_config[DATASET]["MAX_HAUSDORFF"]
ALPHA = config.dataset_config[DATASET]["ALPHA"]
BETA = config.dataset_config[DATASET]["BETA"]
INIT_NUM = config.dataset_config[DATASET]["INIT_NUM"]
EPOCH_START = config.dataset_config[DATASET]["EPOCH_START"]
EPOCH_END = config.dataset_config[DATASET]["EPOCH_END"]
EPOCH_PERIOD = config.dataset_config[DATASET]["EPOCH_PERIOD"]

# define hyperparameters
DEVICE = torch.device("cuda:{:d}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
S_N_EPOCHS = config.dataset_config[DATASET]["training_config"]["S_N_EPOCHS"]
B_N_EPOCHS = config.dataset_config[DATASET]["training_config"]["B_N_EPOCHS"]
T_N_EPOCHS = config.dataset_config[DATASET]["training_config"]["T_N_EPOCHS"]
N_NEIGHBORS = config.dataset_config[DATASET]["training_config"]["N_NEIGHBORS"]
PATIENT = config.dataset_config[DATASET]["training_config"]["PATIENT"]
MAX_EPOCH = config.dataset_config[DATASET]["training_config"]["MAX_EPOCH"]

content_path = CONTENT_PATH
sys.path.append(content_path)

from Model.model import *
net = resnet18()
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

In [4]:
data_provider = DataProvider(content_path, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, verbose=1)

model = SingleVisualizationModel(input_dims=512, output_dims=2, units=256)
negative_sample_rate = 5
min_dist = .1
_a, _b = find_ab_params(1.0, min_dist)
umap_loss_fn = UmapLoss(negative_sample_rate, DEVICE, _a, _b, repulsion_strength=1.0)
recon_loss_fn = ReconstructionLoss(beta=1.0)
criterion = SingleVisLoss(umap_loss_fn, recon_loss_fn, lambd=LAMBDA)

optimizer = torch.optim.Adam(model.parameters(), lr=.01, weight_decay=1e-5)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=.1)


t0 = time.time()
spatial_cons = kcParallelSpatialEdgeConstructor(data_provider=data_provider, init_num=INIT_NUM, s_n_epochs=S_N_EPOCHS, b_n_epochs=B_N_EPOCHS, n_neighbors=N_NEIGHBORS, MAX_HAUSDORFF=MAX_HAUSDORFF, ALPHA=ALPHA, BETA=BETA)

Finish initialization...


In [5]:
s_edge_to, s_edge_from, s_probs, feature_vectors, time_step_nums, selelcted_idxs_list, knn_indices, sigmas, rhos, attention = spatial_cons.construct()

Calculating distances...
calculating distances for 300 points within 0.34 seconds...
Hausdorff distance is 0.29 with 400 points
=================+++=200=+++================
Calculating distances...
calculating distances for 300 points within 0.36 seconds...
Hausdorff distance is 0.30 with 400 points
Finish calculating normaling factor
Calculating distances...
calculating distances for 300 points within 0.37 seconds...
Hausdorff distance is 0.20 with 593 points
select 593 points
Mon Mar 28 15:35:10 2022 Building RP forest with 6 trees
Mon Mar 28 15:35:10 2022 NN descent for 9 iterations
	 1  /  9
	 2  /  9
	 3  /  9
	Stopping threshold met -- exiting after 3 iterations
Gradients calculation: 3.00 seconds	softmax with temperature: 0.00 seconds
=================+++=198=+++================
Calculating distances...
calculating distances for 300 points within 0.36 seconds...
Hausdorff distance is 0.30 with 400 points
Finish calculating normaling factor
Calculating distances...
calculating di

In [13]:
for e in range(80):
    for point in selelcted_idxs_list[e][300:]:
        count=0
        for t in range(80):
            if point in selelcted_idxs_list[t]:
                count = count+1
        print(e,point, count)
        


0 3093 80
0 35311 80
0 26183 80
0 22837 80
0 20506 80
0 4910 80
0 18259 80
0 9215 80
0 18485 80
0 30686 80
0 20205 80
0 35830 80
0 17335 80
0 28553 80
0 31149 80
0 10649 80
0 19173 80
0 48172 80
0 47138 80
0 49263 80
0 4940 80
0 30705 80
0 15419 80
0 29517 80
0 42120 80
0 31008 80
0 7707 80
0 44532 80
0 41789 80
0 40813 80
0 25017 80
0 36973 80
0 13813 80
0 40785 80
0 4895 80
0 17306 80
0 5625 80
0 22434 80
0 17863 80
0 46540 80
0 8693 80
0 26744 80
0 28279 80
0 40740 80
0 16570 80
0 38256 80
0 31223 80
0 49672 80
0 34346 80
0 36113 80
0 3242 80
0 19951 80
0 26645 80
0 3207 80
0 24272 80
0 15819 80
0 45768 80
0 38648 80
0 16546 80
0 12796 80
0 1364 80
0 35606 80
0 5369 80
0 20874 80
0 30134 80
0 15590 80
0 24330 80
0 33786 80
0 10047 80
0 37368 80
0 29122 80
0 25945 80
0 6770 80
0 38660 80
0 48774 80
0 13345 80
0 13934 80
0 41286 80
0 9388 80
0 37868 80
0 27219 80
0 38639 80
0 28618 80
0 5567 80
0 31527 80
0 44344 80
0 2118 80
0 7674 80
0 44333 80
0 13308 80
0 29851 80
0 31786 80
0 301